In [1]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from spatialentropy import altieri_entropy, leibovici_entropy
from scipy.stats import entropy
import numpy as np
import gc
import shapely

from IPython.display import clear_output

from tqdm import tqdm
tqdm.pandas()


In [2]:
gemeenten = gpd.read_parquet("data/gemeenten_amenities.parquet")
wijken = gpd.read_parquet("data/wijken.parquet")

wijken


,wijkcode,wijknaam,gemeentecode,gemeentenaam,IND_WBI,H2O,OAD,STED,BEV_DICHTH,AANT_INW,...,P_GEBBL_EU,P_GEBBL_NE,OPP_TOT,OPP_LAND,OPP_WATER,JRSTATCODE,JAAR,layer,path,geometry
0,WK001400,Centrum,GM0014,Groningen,1.0,NEE,6647.0,1.0,10132.0,23150.0,...,16.0,12.0,241.0,228.0,13.0,2023WK001400,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((6.56023 53.22768, 6.56086 53.22668, ..."
1,WK001401,Oud-Zuid,GM0014,Groningen,3.0,NEE,4222.0,1.0,5212.0,21190.0,...,13.0,9.0,424.0,406.0,18.0,2023WK001401,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((6.58659 53.21063, 6.58589 53.21037, ..."
2,WK001402,Oud-West,GM0014,Groningen,1.0,NEE,5535.0,1.0,12589.0,14155.0,...,12.0,9.0,118.0,112.0,6.0,2023WK001402,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((6.56023 53.22768, 6.55981 53.22759, ..."
3,WK001403,Oud-Noord,GM0014,Groningen,1.0,NEE,5250.0,1.0,10301.0,18300.0,...,9.0,15.0,186.0,178.0,9.0,2023WK001403,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((6.57087 53.22720, 6.56640 53.22533, ..."
4,WK001404,Oosterparkwijk,GM0014,Groningen,1.0,NEE,4369.0,1.0,8552.0,12700.0,...,8.0,13.0,165.0,149.0,16.0,2023WK001404,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((6.58020 53.21707, 6.58015 53.21705, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3319,WK199221,Oostvoorne,GM1992,Voorne aan Zee,2.0,NEE,671.0,4.0,352.0,8390.0,...,3.0,4.0,2853.0,2380.0,472.0,2023WK199221,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((4.10966 51.93562, 4.11267 51.93480, ..."
3320,WK199230,Brielle,GM1992,Voorne aan Zee,2.0,NEE,1111.0,3.0,1817.0,14090.0,...,5.0,5.0,891.0,775.0,116.0,2023WK199230,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((4.15925 51.92780, 4.15928 51.92779, ..."
3321,WK199231,Vierpolders,GM1992,Voorne aan Zee,2.0,NEE,256.0,5.0,188.0,1870.0,...,5.0,3.0,1037.0,992.0,46.0,2023WK199231,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((4.18964 51.90404, 4.19154 51.90337, ..."
3322,WK199232,Zwartewaal,GM1992,Voorne aan Zee,2.0,NEE,254.0,5.0,221.0,1920.0,...,3.0,4.0,952.0,867.0,85.0,2023WK199232,2023,2023 — wijk_2023_v1zw,\\cbsp.nl\Productie\primair\TOP\Werk\KWB_buurt...,"POLYGON ((4.22481 51.86675, 4.21832 51.86395, ..."


: 

In [9]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

def wk_total_amenities_entropy(gm_name, wijkarea, filter_i):
    L0_BLACKLIST, L1_BLACKLIST = ec.getfilter(filter_i)
    amenity_gdf = gpd.read_parquet(f"data/gm_amenities/amenities_{gm_name}.parquet")
    
    # filter out amenities not in the wijk
    amenity_gdf = amenity_gdf[amenity_gdf.within(wijkarea)]
    amenity_gdf.reset_index(drop=True, inplace=True)
    
    # apply filters
    amenity_gdf = amenity_gdf[~amenity_gdf.L0_category.isin(L0_BLACKLIST)]
    if L1_BLACKLIST:
        for key, value in L1_BLACKLIST.items():
            amenity_gdf = amenity_gdf[
                ~(
                    (amenity_gdf.L0_category == key)
                    & (amenity_gdf.L1_category.isin(value))
                )
            ]
    
    # total number of amenities
    total_amenities = len(amenity_gdf)
    
    points = [[point.x, point.y] for point in amenity_gdf.geometry]
    
    # calculate entropy
    L0 = amenity_gdf.loc[:, f"L0_category"].values
    L1 = amenity_gdf.loc[:, f"L1_category"].values
    L0_entropy = _get_shannon_entropy(L0, base=2)
    L1_entropy = _get_shannon_entropy(L1, base=2)
    # L0_entropy = altieri_entropy(points, L0, base=2).entropy
    # L1_entropy = altieri_entropy(points, L1, base=2).entropy
    
    del points
    gc.collect()
    
    return total_amenities, L0_entropy, L1_entropy

In [14]:
for filter in [0, 1, 2]:
    for i, wijk in tqdm(wijken.iterrows(), total=len(wijken)):
        gm_name = wijk.gemeentenaam
        wijkarea = wijk.geometry
        total_amenities, L0_entropy, L1_entropy = wk_total_amenities_entropy(gm_name, wijkarea, filter)
        
        wijken.at[i, f"total_amenities_{filter}"] = total_amenities
        wijken.at[i, f"L0_shannon_{filter}"] = L0_entropy
        wijken.at[i, f"L1_shannon_{filter}"] = L1_entropy
        
        del total_amenities, L0_entropy, L1_entropy
        gc.collect()

wijken.to_parquet("data/wijken_entropy.parquet")

 29%|██▉       | 965/3324 [32:55<1:20:28,  2.05s/it] 


KeyboardInterrupt: 

In [ ]:
wijken